In [32]:
#import pacakges 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image

import math
import numpy
import cv2
from pylab import *
from numpy import *
from numpy import array
import random;

# 1) Use ginput to find correspondances

In [11]:
def getPoints(image):
    img1 = Image.open(image) 
    plt.figure()
    plt.imshow(img1)
    print("Please click") 
    x = plt.ginput(4)
    return x

In [17]:

pts_src = getPoints('./Images/image1.jpg')
print(pts_src)
pts_dst=getPoints('./Images/image2.jpg')
print(pts_dst)

Please click
[(887.73089952429, 328.956851927084), (926.2188495157718, 328.956851927084), (811.6500681457796, 320.0061658825534), (774.0571867587507, 492.75440654199485)]
Please click
[(445.5670089244761, 304.78999960685127), (478.6845472892395, 308.3702740246635), (374.85658917268404, 293.15410774896145), (316.6771298832348, 461.4270053861376)]


In [34]:
img1 = array(Image.open('./Images/image1.jpg')) 
img1.shape

(683, 1024, 3)

#

In [18]:
#get x,y,x',y' arrays from cores
def getArrays(src,dest):
    x=[]
    y=[]
    xdash=[]
    ydash=[]

    for i in range(4):
        x.append(src[i][0])
        y.append(src[i][1])
        xdash.append(dest[i][0])
        ydash.append(dest[i][1])

    return x,y,xdash,ydash

In [19]:
#homography parameters implemntation
def findHomography (src, dest):
    b = dest.flatten().reshape(-1,1)

    x,y,xdash,ydash = getArrays(src,dest)

    M = np.array([
          [x[0],y[0],1,0,0,0,-x[0]*xdash[0], -y[0]*xdash[0]],
          [0,0,0,x[0],y[0],1,-x[0]*ydash[0], -y[0]*ydash[0]],
          [x[1],y[1],1,0,0,0,-x[1]*xdash[1],-y[1]*xdash[1]],
          [0,0,0,x[1],y[1],1,-x[1]*ydash[1],-y[1]*ydash[1]],
          [x[2],y[2],1,0,0,0,-x[2]*xdash[2],-y[2]*xdash[2]],
          [0,0,0,x[2],y[2],1,-x[2]*ydash[2],-y[2]*ydash[2]],
          [x[3],y[3],1,0,0,0,-x[3]*xdash[3],-y[3]*xdash[3]],
          [0,0,0,x[3],y[3],1,-x[3]*ydash[3],-y[3]*ydash[3]]
      ])
    a = np.dot(np.linalg.inv(M),b)
    a = np.append(a,1)
    return a.reshape(3,3)

In [48]:
#test to check homography function
pts_src_test = np.array([[141, 131], [480, 159], [493, 630],[64, 601]])
pts_dst_test = np.array([[318, 256],[534, 372],[316, 670],[73, 473]])
h_test=findHomography(pts_src_test,pts_dst_test)
print("calculated homography", h_test)
h_test, status_tst = cv2.findHomography(pts_src_test, pts_dst_test)
print("actual homogrpahy", h_test)


calculated homography [[ 4.34043935e-01 -4.19622184e-01  2.91709494e+02]
 [ 1.46491654e-01  4.41418278e-01  1.61369294e+02]
 [-3.62463336e-04 -9.14274844e-05  1.00000000e+00]]
actual homogrpahy [[ 4.34043935e-01 -4.19622184e-01  2.91709494e+02]
 [ 1.46491654e-01  4.41418278e-01  1.61369294e+02]
 [-3.62463336e-04 -9.14274844e-05  1.00000000e+00]]


In [21]:
h=findHomography(np.array(pts_src),np.array(pts_dst))
print("calculated homography", h)


calculated homography [[ 7.65795701e-01 -7.42374428e-01 -1.37949172e+02]
 [ 1.85161387e-01 -3.22770121e-01  1.45419697e+02]
 [ 3.98926141e-04 -2.08564284e-03  1.00000000e+00]]


# Warping

# Forward

In [104]:
def Get_new_boundaries (h,M,N):
    A_new  = np.matmul(h,[0,0,1] )
    A_new = correct(A_new)
    #A_new = round_arr(A_new)
    B_new = np.matmul(h,[0,N-1,1] )
    B_new = correct(B_new)
   # B_new = round_arr(B_new)
    C_new = np.matmul(h,[M-1,0,1] )
    C_new = correct(C_new)
    #C_new = round_arr(C_new)
    D_new = np.matmul(h,[M-1,N-1,1] )
    D_new = correct(D_new)
    #D_new = round_arr(D_new)
    print(A_new,B_new,C_new,D_new)
    return A_new,B_new,C_new,D_new

In [72]:
def correct(A):
    if A[2]!=1:
        A[0] = (A[0]/A[2])
        A[1] = (A[1]/A[2])
        A[2] = (A[2]/A[2])
    return A
def round_arr(A):
    A[0]= round(A[0])
    A[1]= round(A[1])
    A[2]= round(A[2])
    return A  
    
        

In [40]:
def Get_max_min_Axis (A,B,C,D):
    
    min_x = min(A[0],B[0],C[0],D[0])
    max_x = max(A[0],B[0],C[0],D[0])
    min_y = min(A[1],B[1],C[1],D[1])
    max_y = max(A[1],B[1],C[1],D[1])
   
    return min_x , max_x , min_y , max_y

In [107]:
A,B,C,D = Get_new_boundaries(h,img1.shape[0],img1.shape[1])

[-137.94917165  145.41969672    1.        ] [791.62686483 162.9958349    1.        ] [302.12505052 213.58908669   1.        ] [435.41027238  67.89438823   1.        ]


In [118]:
min_x , max_x , min_y, max_y = Get_max_min_Axis (A,B,C,D)
print(min_x , max_x , min_y, max_y)

-137.9491716491757 791.6268648280867 67.89438822914488 213.5890866946835


In [119]:
new_w = int(abs(max_x - min_x))
new_h = int(abs(max_y - min_y))
print(new_w,new_h)

929 145


In [117]:
h_inv = np.linalg.inv(h)
#h_inv

In [120]:
n=np.zeros((new_w,new_h))
print(n.shape)
print(img1.shape)

(929, 145)
(683, 1024, 3)


In [ ]:
gray= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

In [112]:
co_ord = np.matmul(h_inv,D)
print(correct(co_ord))

[6.820e+02 1.023e+03 1.000e+00]


In [ ]:
new_img = np.zeros((new_w,new_h))
for i in range (min_x,(min_x+new_w),1):
    for j in range (min_y,(min_y+new_h),1):
                   point = [i,j,1]
                   co_ord = np.matmul(h_inv,point)
                   co_ord = correct(co_ord)
                   
                
